## License

Copyright 2019 Ruud van Deursen, Firmenich SA.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

## Example Notebook

This is an example notebook to model graphs using generative examinaton networks.

In [ ]:
#!pip install livelossplot
#!pip install networkx
from lgi_generative_model_utils import DataUtils
import keras.backend as K
from lgi_generative_model_alt import *
import tensorflow as tf
from numpy import arange
config = None
from lgi_valid_graph import GraphValidator
#from livelossplot.tf_keras import PlotLossesCallback

### Define memory block

In [ ]:
if config is None:
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.log_device_placement = True
    sess = tf.Session(config=config)
    K.set_session(sess)

### Read data

In [ ]:
# Read the data and setup Utils
trainingset = "datasets/Graphs_rnd_aug_1x.lgi"
Utils = DataUtils(maxlen=42,step=3) # This defines graphs with 25 vertices

# Prepare data with an augmentation method
text = Utils.Prepare(trainingset)
print("Evaluated: %s"%(Utils.evaluated))
print("Kept entries: %s"%(Utils.kept))
print("Corpus length: %s"%(len(text)))
print("Total chars %s"%(len(Utils.okchars)))
print("Max lenghth %s"%(Utils.maxg6))

# Translate to X,Y and get maxlen observed
# We take the computed maxlen, because
# this max very on the dataset used.
X,y,maxlen = Utils.Encode(text)

### Methods used for modeling

In [ ]:
def IsValidGraph(lgi):
    return GraphValidator.IsValid(lgi)
    
def augment(lgi,naug=10):
    return GraphRandomizer.Augment(lgi)

In [ ]:
IsValidGraph("B1BBBB1")

### Setup the model

In [ ]:
num_iterations,batch_size = 80,1024
def TestModel(Utils,l1=64,l2=64,Bidirectional=[True,False],nmodels=4,merge=0):
    model_setup = BaseModel(Utils,Layers=[l1,l2],Bidirectional=Bidirectional,minimodels=nmodels,merge=merge)
    model = model_setup.Init(verbose=True)
    return model_setup

# Define num iterations
def RunModel(model_setup,filepath,verbose=0):
    logfile = "%s.csv"%(filepath[:filepath.index("-cuda5")])
    trainer = model_setup.InitTrainer(sanitycheck=IsValidGraph,verbose=True,minimum=.97,patience=10)
    model,history = trainer.Fit(X,y,filepath,logfile,num_epochs=num_iterations,batch_size=batch_size,
                                mycallbacks=[],verbose=verbose)
    return (model,history)

In [ ]:
l1,l2 = 128,64
nmodels = 4
b1,b2 = False,False
suffices = {0:"Concatenated"}
#suffices = {0:"Concatenated",1:"Averaged",2:"Learnable-average"}
model_setup = dict([(merge,TestModel(Utils,l1=l1,l2=l2,Bidirectional=[b1,b2],merge=merge,nmodels=nmodels)) for merge in suffices.keys()])

### Run the training

In [ ]:
merge,run = 0,0 # Merge concatenated
filepath = "test_models/Graphs_rnd_1x_%s-%s_lstm-4xlstm_%s_run_%s-cuda5-sg-{epoch:03d}-{loss:.4f}.hdf5"%(l1,l2,suffices[merge],run)
models,history = RunModel(model_setup[merge],filepath,verbose=1)    

In [ ]:
merge,run = 0,1 # Merge concatenated
filepath = "test_models/Graphs_rnd_1x_%s-%s_lstm-4xlstm_%s_run_%s-cuda5-sg-{epoch:03d}-{loss:.4f}.hdf5"%(l1,l2,suffices[merge],run)
models,history = RunModel(model_setup[merge],filepath,verbose=1)    

In [ ]:
merge,run = 0,2 # Merge concatenated
filepath = "test_models/Graphs_rnd_1x_%s-%s_lstm-4xlstm_%s_run_%s-cuda5-sg-{epoch:03d}-{loss:.4f}.hdf5"%(l1,l2,suffices[merge],run)
models,history = RunModel(model_setup[merge],filepath,verbose=1)    

### Generate samples for evaluation

In [ ]:
fname = dict([
    ("<model path>",False),
    ("<model path>",False),
    ("<model path>",False),
]) 

In [ ]:
from datetime import datetime
for file in fname.keys():
    gen = model_setup[0].InitGenerator("test_models/%s"%(file))
    fout = file.replace("test_models","generated").replace(".hdf5","_%09d_%09d.lgi")
    start,batch,end = 0,180,540
    while start < end:       
        now = datetime.now()
        graphs = gen.Predict(ncollect=batch)
        now2 = datetime.now()
        f = fout%(start,start+batch)
        with open("generated_results/%s"%(f),"w") as ios:
            _ = [ios.write("%s\n"%(g6)) for g6 in graphs]
        now3 = datetime.now()
        diff1 = now2-now
        diff2 = now3-now2
        diff1 = diff1.seconds + 1e-6*diff1.microseconds
        diff2 = diff2.seconds + 1e-6*diff2.microseconds
        start += batch
        print("Written %s Gen=%.3f Writing=%.3f"%(f,diff1,diff2))

In [ ]:
from datetime import datetime
for file in fname.keys():
    gen = model_setup[0].InitGenerator("test_models/%s"%(file))
    fout = file.replace("test_models","generated").replace(".hdf5","_%09d_%09d.lgi")
    start,batch,end = 0,2000,6000
    while start < end:       
        now = datetime.now()
        graphs = gen.Predict(ncollect=batch)
        now2 = datetime.now()
        f = fout%(start,start+batch)
        with open("generated_results/%s"%(f),"w") as ios:
            _ = [ios.write("%s\n"%(g6)) for g6 in graphs]
        now3 = datetime.now()
        diff1 = now2-now
        diff2 = now3-now2
        diff1 = diff1.seconds + 1e-6*diff1.microseconds
        diff2 = diff2.seconds + 1e-6*diff2.microseconds
        start += batch
        print("Written %s Gen=%.3f Writing=%.3f"%(f,diff1,diff2))

### Generate many

In [ ]:
from datetime import datetime
for file in fname.keys():
    gen = model_setup[0].InitGenerator("test_models/%s"%(file))
    fout = file.replace("test_models","generated").replace(".hdf5","_%09d_%09d.lgi")
    start,batch,end = 250000,10000,1e7
    while start < end:       
        now = datetime.now()
        graphs = gen.Predict(ncollect=batch)
        now2 = datetime.now()
        f = fout%(start,start+batch)
        with open("time_series/%s"%(f),"w") as ios:
            _ = [ios.write("%s\n"%(g6)) for g6 in graphs]
        now3 = datetime.now()
        diff1 = now2-now
        diff2 = now3-now2
        diff1 = diff1.seconds + 1e-6*diff1.microseconds
        diff2 = diff2.seconds + 1e-6*diff2.microseconds
        start += batch
        print("Written %s Gen=%.3f Writing=%.3f"%(f,diff1,diff2))
    break